# 3_training_mlflow.ipynb

- **Objectif** :
Lancer et suivre les entraînements des modèles ML (Logistic, Random Forest, XGBoost)
avec différentes stratégies de déséquilibre, via MLflow.
---

In [ ]:
import os
import mlflow
import mlflow.sklearn
from IPython.display import Markdown as md

## Définir le dossier du projet

In [ ]:
# Définir le dossier du projet
os.chdir("..") if "notebooks" in os.getcwd() else None

## Configurer le tracking local MLflow

In [ ]:
# Configurer le tracking local MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("Churn_Prediction_Models")

In [ ]:
md("## 🚀 Lancement des entraînements et suivi MLflow")

## 1️⃣ Lancer les scripts

In [ ]:
# ==========================================================
# 1️⃣ Lancer les scripts
# ==========================================================
import subprocess

In [ ]:
scripts = [
    "scripts/logistic_script.py",
    "scripts/forest_script.py",
    "scripts/xgboost_script.py"
]

In [ ]:
for script in scripts:
    print(f"⚙️ Entraînement avec {script} ...")
    subprocess.run(["python", script])
    print(f"✅ Terminé : {script}")

## 2️⃣ Comparaison des résultats

In [ ]:
# ==========================================================
# 2️⃣ Comparaison des résultats
# ==========================================================
md("### 📊 Visualisation dans MLflow UI")

In [ ]:
print("""
Pour ouvrir l'interface MLflow :
> mlflow ui --backend-store-uri ./mlruns
Puis ouvrir : http://127.0.0.1:5000
""")

In [ ]:
md("### 🏆 Sélection du meilleur modèle")

In [ ]:
import pandas as pd

## Charger les runs MLflow

In [ ]:
# Charger les runs MLflow
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("Churn_Prediction_Models")
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

In [ ]:
best_run = runs.sort_values("metrics.f1_score", ascending=False).iloc[0]
md(f"**Meilleur modèle :** {best_run['tags.model_name']} (F1 = {best_run['metrics.f1_score']:.3f})")

## Chargement et test du modèle

In [ ]:
# Chargement et test du modèle
model_uri = f"runs:/{best_run['run_id']}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [ ]:
md("### 🔍 Test d'une prédiction avec le modèle chargé")
import joblib
import pandas as pd

## Exemple d’un batch de test

In [ ]:
# Exemple d’un batch de test
X_test = joblib.load("data/X_test.pkl")
print("✅ Prédiction exemple :")
print(loaded_model.predict(X_test[:5]))